# VacationPy
---

### Step 0: Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Susuman,62.78075,148.15396,7.67,74,99,1.22,RU,1719658291
1,1,Puerto Natales,-51.72987,-72.50603,-3.33,93,100,1.43,CL,1719658293
2,2,Sinabang,2.48030,96.38010,25.83,84,100,1.53,ID,1719658294
3,3,Anadyr,64.73424,177.51030,6.08,93,20,7.00,RU,1719658295
4,4,Puerto Ayora,-0.74018,-90.31380,21.17,100,75,3.04,EC,1719658296


---
### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point is the humidity in each city.

In [104]:
%%capture --no-display
# Configure the map plot
map_cities = city_data_df.hvplot.points(
    "Lng", "Lat", frame_width = 700, frame_height = 500,
    bgcolor='#ccc', geo = True, coastline = True,
    tiles = "EsriImagery", tiles_opts = {'alpha': 0.1},
    size = "Humidity", scale = 0.8, color='Max Temp', cmap='cet_rainbow4', alpha=0.6
)
# Display the map
map_cities

:Overlay
   .WMTS.I      :WMTS   [Longitude,Latitude]
   .Points.I    :Points   [Lng,Lat]   (Max Temp,Humidity)
   .Coastline.I :Feature   [Longitude,Latitude]

---
### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [63]:
# Narrow down cities that fit criteria and drop any results with null values
temp = [22,27]
humd = [0,80]
city_ideal_df = city_data_df[(city_data_df['Max Temp']>temp[0])&(city_data_df['Max Temp']<temp[1])&(city_data_df['Humidity']>humd[0])&(city_data_df['Humidity']<humd[1])]
# Drop any rows with null values
city_ideal_df = city_ideal_df.dropna()
# Display sample data
city_ideal_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,Bredasdorp,-34.53215,20.04031,24.80,36,54,4.54,ZA,1719658306
26,26,Port Alfred,-33.58601,26.88329,26.77,41,67,2.24,ZA,1719658322
28,28,Skellefteå,64.75067,20.95279,26.66,53,75,7.20,SE,1719658325
30,30,Uturoa,-16.72891,-151.44482,26.35,76,5,11.57,PF,1719658327
36,36,Palangkaraya,-2.20833,113.91667,26.01,75,61,1.79,ID,1719658334


---
### Step 3: Create a new DataFrame called `hotel_df`.

In [72]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_ideal_df[['City','Country','Lat','Lng','Humidity','Max Temp']].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''
# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Max Temp,Hotel Name
12,Bredasdorp,ZA,-34.53215,20.04031,36,24.80,
26,Port Alfred,ZA,-33.58601,26.88329,41,26.77,
28,Skellefteå,SE,64.75067,20.95279,53,26.66,
30,Uturoa,PF,-16.72891,-151.44482,76,26.35,
36,Palangkaraya,ID,-2.20833,113.91667,75,26.01,


---
### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [73]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "filter":'',
    "limit":20,
    "bias":'',
    "apiKey":geoapify_key,
    "categories": "accommodation.hotel"
}
# Print a message to follow up the hotel search
print("Starting hotel search")
# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    params["categories"] = "accommodation.hotel"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    # Make and API request using the params dictionary and convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found try other accomodations, if nothing, set the hotel name as "No hotel found".
        params["categories"] = "accommodation"
        name_address = requests.get(base_url, params=params).json()
        try: 
            hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        except (KeyError, IndexError):
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
# Display sample data
hotel_df.head()

Starting hotel search
Bredasdorp - nearest hotel: Victoria Hotel
Port Alfred - nearest hotel: No hotel found
Skellefteå - nearest hotel: Hotel Victoria
Uturoa - nearest hotel: Hinano
Palangkaraya - nearest hotel: Hotel NEO
Udachny - nearest hotel: Вилюй
Touros - nearest hotel: Pousada Atlântico
Yemva - nearest hotel: Гостиница Континент
Luabo - nearest hotel: No hotel found
Tsiombe - nearest hotel: No hotel found
Saint-Philippe - nearest hotel: Le Baril
Georgetown - nearest hotel: No hotel found
Es Castell - nearest hotel: Sa Casa
Hermanus - nearest hotel: Aloe guest house
Port Mathurin - nearest hotel: Escale Vacances
Port Hedland - nearest hotel: All Seasons Hotel
Laojunmiao - nearest hotel: No hotel found
Cabo San Lucas - nearest hotel: Comfort Rooms
Beloha - nearest hotel: No hotel found
Porto Novo - nearest hotel: Por do Sol
Orange Cove - nearest hotel: No hotel found
Borgo - nearest hotel: Castellu
Inhambane - nearest hotel: Hotel de Inhambane
Yermentau - nearest hotel: Достык
Gu

,City,Country,Lat,Lng,Humidity,Max Temp,Hotel Name
12,Bredasdorp,ZA,-34.53215,20.04031,36,24.80,Victoria Hotel
26,Port Alfred,ZA,-33.58601,26.88329,41,26.77,No hotel found
28,Skellefteå,SE,64.75067,20.95279,53,26.66,Hotel Victoria
30,Uturoa,PF,-16.72891,-151.44482,76,26.35,Hinano
36,Palangkaraya,ID,-2.20833,113.91667,75,26.01,Hotel NEO


---
### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [74]:
# Import ISO country names
country_names = pd.read_csv('resources/countries_iso3166b.csv', encoding_errors='ignore')
country_names = country_names.rename(columns={'iso2':'Country Code','country_common':'Country'})[['Country Code','Country']]
country_names.head()

,Country Code,Country
0,AF,Afghanistan
1,AX,Aland Islands
2,AL,Albania
3,DZ,Algeria
4,AS,American Samoa


In [75]:
has_hotel_df = hotel_df[hotel_df['Hotel Name']!='No hotel found'].rename(columns={'Country':'Country Code'}).merge(country_names,on='Country Code')
has_hotel_df.head()

,City,Country Code,Lat,Lng,Humidity,Max Temp,Hotel Name,Country
0,Bredasdorp,ZA,-34.53215,20.04031,36,24.80,Victoria Hotel,South Africa
1,Skellefteå,SE,64.75067,20.95279,53,26.66,Hotel Victoria,Sweden
2,Uturoa,PF,-16.72891,-151.44482,76,26.35,Hinano,French Polynesia
3,Palangkaraya,ID,-2.20833,113.91667,75,26.01,Hotel NEO,Indonesia
4,Udachny,RU,66.42989,112.40210,34,26.02,Вилюй,Russian Federation


In [106]:
%%capture --no-display
# Configure the map plot
map_hotels = has_hotel_df.hvplot.points(
    "Lng", "Lat", frame_width = 700, frame_height = 500,
    bgcolor='#eee', geo = True, coastline = True,
    tiles = "EsriImagery", tiles_opts = {'alpha': 0.05},
    size = "Humidity", scale = 0.8, color='Max Temp', cmap='cet_rainbow', alpha=0.8,
    hover_cols=['Hotel Name','City','Country']
)
# Display the map
map_hotels

:Overlay
   .WMTS.I      :WMTS   [Longitude,Latitude]
   .Points.I    :Points   [Lng,Lat]   (Max Temp,Humidity,Hotel Name,City,Country)
   .Coastline.I :Feature   [Longitude,Latitude]